

<table>
    <center>
            <img src="https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/snowflake.png">
    </center>
        <td>
            <img src="../Images/logo.jpg" width="300">
        </td>
        <td>
            <img src="../Images/call_center.jpg" width="300">
        </td>
</table>

# Call Database to Snowflake

### Make sure you are running numpy 1.18.5
Tensorflow and Keras will throw an error with numpy otherwise

In [ ]:
## Make sure you are in the right conda env
!conda env list

In [ ]:
!python --version

In [ ]:
## If you are having problems with the conda env:
## Execute these in the terminal, respond with 'y' when prompted
# !conda install keras
# !conda install seaborn
# !pip install -U numpy==1.18.5

In [ ]:
# !python -m pip install --upgrade pip
# !pip install -r https://raw.githubusercontent.com/snowflakedb/snowflake-connector-python/v2.4.6/tested_requirements/requirements_36.reqs
# !pip install snowflake-connector-python
# !pip install snowflake-connector-python[pandas]

In [ ]:
# Make sure to pip the connector and install the dependencies:
# https://docs.snowflake.com/en/user-guide/python-connector-install.html
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.regularizers import l1
from datetime import timedelta

try:
    f = open("../Login_Credentials/SF_Account_Info.txt", "r")
except:
    f = open("..\Login_Credentials\SF_Account_Info.txt", "r")
    
usr = f.readline()
usr = usr.replace('\n', '')
psswrd = f.readline()
psswrd = psswrd.replace('\n', '')
accnt = f.readline()
accnt = accnt.replace('\n', '')
dbName = f.readline()
dbName = dbName.replace('\n', '')
path = f.readline()
path = path.replace('\n', '')
file1 = f.readline()
file1 = file1.replace('\n', '')
file2 = f.readline()
file2 = file2.replace('\n', '')
file3 = f.readline()
file3 = file3.replace('\n', '')
file4 = f.readline()
file4 = file4.replace('\n', '')
file5 = f.readline()
file5 = file5.replace('\n', '')
file6 = f.readline()
file6 = file6.replace('\n', '')
file7 = f.readline()
file7 = file7.replace('\n', '')
file8 = f.readline()
file8 = file8.replace('\n', '')

In [ ]:
try:
    print(usr,'\n', psswrd,'\n', accnt,'\n', dbName, '\n', 
          path, '\n', file1,'\n', file2,'\n', file3, '\n', 
          file4,'\n', file5,'\n', file6,'\n', file7,'\n', file8)
except:
    print('Check Login_Credentials/README.md for more info.')

In [ ]:
try:
    f1 = open("../Login_Credentials/SHARED_SF_Account_Info.txt", "r")
except:
    f1 = open("..\Login_Credentials\SHARED_SF_Account_Info.txt", "r")
    
shareName = f1.readline()
shareName = shareName.replace('\n', '')

sharedAccnt1 = f1.readline()
sharedAccnt1 = sharedAccnt1.replace('\n', '')
sharedAccnt2 = f1.readline()
sharedAccnt2 = sharedAccnt2.replace('\n', '')
sharedAccnt3 = f1.readline()
sharedAccnt3 = sharedAccnt3.replace('\n', '')
sharedAccnt4 = f1.readline()
sharedAccnt4 = sharedAccnt4.replace('\n', '')
sharedAccnt5 = f1.readline()
sharedAccnt5 = sharedAccnt5.replace('\n', '')
sharedAccnt6 = f1.readline()
sharedAccnt6 = sharedAccnt6.replace('\n', '')
sharedAccnt7 = f1.readline()
sharedAccnt7 = sharedAccnt7.replace('\n', '')
sharedAccnt8 = f1.readline()
sharedAccnt8 = sharedAccnt8.replace('\n', '')
sharedAccnt9 = f1.readline()
sharedAccnt9 = sharedAccnt9.replace('\n', '')
sharedAccnt10 = f1.readline()
sharedAccnt10 = sharedAccnt10.replace('\n', '')
sharedAccnt11 = f1.readline()
sharedAccnt11 = sharedAccnt11.replace('\n', '')
sharedAccnt12 = f1.readline()
sharedAccnt12 = sharedAccnt12.replace('\n', '')
sharedAccnt13 = f1.readline()
sharedAccnt13 = sharedAccnt13.replace('\n', '')
sharedAccnt14 = f1.readline()
sharedAccnt14 = sharedAccnt14.replace('\n', '')
sharedAccnt15 = f1.readline()
sharedAccnt15 = sharedAccnt15.replace('\n', '')

try:
    print(f'The Following Share Will Be Created If It Does Not Exist: {shareName}\n' 
          + 'The Following Accounts Will Be Granted Share Access:\n'
          + sharedAccnt1, sharedAccnt2, sharedAccnt3, sharedAccnt4,
          sharedAccnt5, sharedAccnt6, sharedAccnt7, sharedAccnt8, 
          sharedAccnt9, sharedAccnt10, sharedAccnt11, sharedAccnt12,
          sharedAccnt13, sharedAccnt14, sharedAccnt15)
except:
    print('Check Login_Credentials/README.md for more info.')

In [ ]:
accountList = [sharedAccnt1, sharedAccnt2, sharedAccnt3, sharedAccnt4, sharedAccnt5,
               sharedAccnt6, sharedAccnt7, sharedAccnt8, sharedAccnt9, sharedAccnt10,
               sharedAccnt11, sharedAccnt12, sharedAccnt13, sharedAccnt14, sharedAccnt15]
ACCOUNT_LIST = []
for i in accountList:
    if len(i) != 0:
        ACCOUNT_LIST.append(i)

In [ ]:
aSQLstatement = 'USE database {};'.format(dbName)
sqlScript = 'create share {};'.format(shareName)
sqlScript1 = 'grant usage on database {} to share {};'.format(dbName, shareName)
sqlScript2 = 'grant usage on schema {}.public to share {};'.format(dbName, shareName)
sqlScript3 = 'grant select on all tables in schema {}.public to share {};'.format(dbName, shareName)

In [ ]:
# Connect to SF
conn = snowflake.connector.connect(
    user=usr,
    password=psswrd,
    account=accnt,
    role='ACCOUNTADMIN',
    warehouse='COMPUTE_WH',
    #database='<SOME_DATABASE>',
    #table='<SOME_TABLE>',
    #schema='PUBLIC'
                                    )
print("Connected to Snowflake using " + usr + " " + accnt)

In [ ]:
os.chdir(path)
print("\tUsing files from dir " + path)

In [ ]:
def uploadExcel(fileName):
    # Returns Year of File Passed to Method
    def fileYear():
        aYear = fileName[-7:-5]
        return '20' + aYear
    
    # Creates String to Pass to SF Python Connector for Table Creation
    def moreFormat(theInput):
        
        anthrList = []
        stringList = []
        o = 0
        
        for i in theInput:
            if i.startswith(' '):
                theInput[o] = i.strip()
            o = o + 1
            
            if len(i) > 255:
                i = i[:255]
        
            splChar = [
            '(',')','[',']','/','\\','-','.',':','\t','<','>','"','?',
            '!','`',"'",'&','^',' ','{','}','+','=','~','|',';','%','#']
            for x in splChar:
                i = i.replace(x, '_')
            anthrList.append(i)
            stringList.append(i)

        stringList = str(stringList)
        stringList = stringList.replace("'", '"')
        stringList = stringList.replace(",", ' varchar,\n')
        stringList = stringList.strip("[]")
        stringList = stringList + " varchar"

        return anthrList, stringList
    
    # Convert to Pandas DF
    def makePandas():
        if fileName != '':
            dfSheet1 = pd.read_excel(fileName, sheet_name=0)
            dfSheet2 = pd.read_excel(fileName, sheet_name=1)
            dfSheet3 = pd.read_excel(fileName, sheet_name=2)
            dfSheet1.convert_dtypes(), dfSheet2.convert_dtypes(), dfSheet3.convert_dtypes()
            
            # Call moreFormat to pass string to SF Python Connector & Change DF Column Names to Match
            thisVar1 = dfSheet1.columns.tolist()
            dfSheet1.columns, stringList1 = moreFormat(thisVar1)
            thisVar2 = dfSheet2.columns.tolist()
            dfSheet2.columns, stringList2 = moreFormat(thisVar2)
            thisVar3 = dfSheet3.columns.tolist()
            dfSheet3.columns, stringList3 = moreFormat(thisVar3)
            
            # Create Cursor Object
            conn.cursor().execute("CREATE DATABASE IF NOT EXISTS {}".format(dbName.upper()))
            conn.cursor().execute("USE DATABASE {}".format(dbName.upper()))
            
            # Create Snowflake Table and Columns & Upload the DF to SF
            conn.cursor().execute(
                "CREATE OR REPLACE TABLE "
                "NEEDSMETANDUNMET{}({})".format(fileYear(), stringList1))
            success, nchunks, nrows, _ = write_pandas(conn, dfSheet1, 'NEEDSMETANDUNMET{}'.format(fileYear()))
            
            conn.cursor().execute(
                "CREATE OR REPLACE TABLE "
                "CALLREPORTS{}({})".format(fileYear(), stringList2))
            success, nchunks, nrows, _ = write_pandas(conn, dfSheet2, 'CALLREPORTS{}'.format(fileYear()))
            
            conn.cursor().execute(
                "CREATE OR REPLACE TABLE "
                "REFERRALS{}({})".format(fileYear(), stringList3))
            success, nchunks, nrows, _ = write_pandas(conn, dfSheet3, 'REFERRALS{}'.format(fileYear()))            
             
            return print(fileName + ' Uploaded Succesfully To Snowflake!')
        else:
            return print('Nothing More to Upload.')
    return makePandas()    

In [ ]:
print("Uploading " + file8 + "\nThis will take several minutes.")
uploadExcel(file8)

In [ ]:
print("2021 Data File Provided in SF_Account_Info.txt have been uploaded now.")

## SQL to be Executed on Snowflake
Preprocessing

In [ ]:
# ---------------------------------------------------------------------------
# -- SPECIFICIES CORRECT SNOWFLAKE SETTINGS
# ---------------------------------------------------------------------------
sqlScript4 = 'USE role ACCOUNTADMIN;'
sqlScript5 = 'USE warehouse COMPUTE_WH;'
sqlScript6 = 'USE database "{}";'.format(dbName)

# ---------------------------------------------------------------------------
# -- CREATES TABLE AND ADDS ALL YEARS DATA TO IT
# ---------------------------------------------------------------------------
# --DROP TABLE "TOTAL_NEEDSMETANDUNMET"
sqlScript7 = 'INSERT INTO "TOTAL_NEEDSMETANDUNMET" SELECT "DateOfCall", "CallReportNum", "TaxonomyCode", "TaxonomyName", "StateProvince" FROM "NEEDSMETANDUNMET2021" WHERE "NEEDSMETANDUNMET2021"."DateOfCall" NOT IN (SELECT "DateOfCall" FROM "TOTAL_NEEDSMETANDUNMET");'

# ---------------------------------------------------------------------------
# -- CREATES TABLE AND ADDS ALL YEARS DATA TO IT
# ---------------------------------------------------------------------------
# --DROP TABLE "ADBNAMEHERE"."PUBLIC"."TOTAL_CALLREPORTS"
sqlScript15 = 'INSERT INTO "TOTAL_CALLREPORTS" SELECT "CallReportNum", "Demographics___Gender__Person_in_Need_", "Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_" FROM "CALLREPORTS2021" WHERE "CALLREPORTS2021"."DateOfCall" NOT IN (SELECT "DateOfCall" FROM "TOTAL_CALLREPORTS");'
# ---------------------------------------------------------------------------
# -- CREATES TABLE FROM TOTAL_* TABLES 13 COLUMNS TOTAL
# ---------------------------------------------------------------------------
# --DROP TABLE "TOTAL_SATURNCLOUD" 
sqlScript23 = 'CREATE TABLE "TOTAL_SATURNCLOUD" AS SELECT "TOTAL_NEEDSMETANDUNMET"."DateOfCall", "TOTAL_NEEDSMETANDUNMET"."CallReportNum", "TOTAL_NEEDSMETANDUNMET"."TaxonomyCode", "TOTAL_NEEDSMETANDUNMET"."TaxonomyName", "TOTAL_NEEDSMETANDUNMET"."StateProvince", "TOTAL_CALLREPORTS"."Demographics___Gender__Person_in_Need_", "TOTAL_CALLREPORTS"."Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_" FROM "ADBNAMEHERE"."PUBLIC"."TOTAL_NEEDSMETANDUNMET" INNER JOIN "TOTAL_CALLREPORTS" ON "TOTAL_NEEDSMETANDUNMET"."CallReportNum"="TOTAL_CALLREPORTS"."CallReportNum";'

# ---------------------------------------------------------------------------
# -- REMOVE ALL NON CA VALUES FROM STATEPROVINCE AND THEN DROPS THE COL
# ---------------------------------------------------------------------------
sqlScript24 = 'DELETE FROM "TOTAL_SATURNCLOUD" WHERE "StateProvince" IS NULL;'
sqlScript25 = 'DELETE FROM "TOTAL_SATURNCLOUD" WHERE "StateProvince"!=\'CA\';'
sqlScript26 = 'ALTER TABLE "TOTAL_SATURNCLOUD" DROP COLUMN "StateProvince";'

# ---------------------------------------------------------------------------
# -- REMOVE ALL REMAINING NULL VALUES FROM THE REST OF THE COLUMNS
# ---------------------------------------------------------------------------
sqlScript27 = 'DELETE FROM "TOTAL_SATURNCLOUD" WHERE "Demographics___Gender__Person_in_Need_" IS NULL;'
sqlScript28 = 'DELETE FROM "TOTAL_SATURNCLOUD" WHERE "Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_" IS NULL;'

# ---------------------------------------------------------------------------
# -- REMOVE ALL DUPLICATE ROWS WITH SAME CALLREPORTNUM AND TAXONOMY CODE
# ---------------------------------------------------------------------------
sqlScript29 = 'DELETE FROM "TOTAL_SATURNCLOUD" WHERE "TaxonomyCode" NOT LIKE \'R%\';'

# ---------------------------------------------------------------------------
# -- REMOVE ALL DUPLICATE ROWS WITH SAME CALLREPORTNUM AND TAXONOMY CODE
# ---------------------------------------------------------------------------
sqlScript30 = 'CREATE OR REPLACE TABLE "TOTAL_SATURNCLOUD" AS SELECT DISTINCT * FROM "TOTAL_SATURNCLOUD";'

# ---------------------------------------------------------------------------
# -- CONSOLIDATE TAXONOMY CODES WITH * TAG
# ---------------------------------------------------------------------------
sqlScript31 = 'UPDATE "TOTAL_SATURNCLOUD" SET "TaxonomyCode" = LEFT("TaxonomyCode", CHARINDEX(\'*\', "TaxonomyCode") - 2) WHERE CHARINDEX(\'*\', "TaxonomyCode") > 0;'
sqlScript32 = 'UPDATE "TOTAL_SATURNCLOUD" SET "TaxonomyName" = LEFT("TaxonomyName", CHARINDEX(\'*\', "TaxonomyName") - 2) WHERE CHARINDEX(\'*\', "TaxonomyName") > 0;'

# ---------------------------------------------------------------------------
# -- FORMAT DATEOFCALL TRUNCATE TIME
# ---------------------------------------------------------------------------
sqlScript33 = 'UPDATE "TOTAL_SATURNCLOUD" SET "DateOfCall" = LEFT("DateOfCall", CHARINDEX(\' \', "DateOfCall") - 1) WHERE CHARINDEX(\' \', "DateOfCall") > 0;'

# ---------------------------------------------------------------------------
# -- Demographics_Prior_or_Current_U_S_Military_Service_Person...
# -- CHANGE ALL NON YES RESPONSES TO NO
# ---------------------------------------------------------------------------
sqlScript34 = 'UPDATE "TOTAL_SATURNCLOUD" SET "Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_"=REPLACE("Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_", \'Unknown\', \'No\');'
sqlScript35 = 'UPDATE "TOTAL_SATURNCLOUD" SET "Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_"=REPLACE("Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_", \'Not Asked\', \'No\');'
sqlScript36 = 'UPDATE "TOTAL_SATURNCLOUD" SET "Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_"=REPLACE("Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_", \'Declined to Answer\', \'No\');'

# ---------------------------------------------------------------------------
# -- Demographics_Gender_Person_in_Need CHANGE NOT ASKED, TRANS, AND
# -- DECLINED TO ANSWER TO OTHER/UNKNOWN/CANNOT DETERMINE
# ---------------------------------------------------------------------------
sqlScript37 = 'UPDATE "TOTAL_SATURNCLOUD" SET "Demographics___Gender__Person_in_Need_"=REPLACE("Demographics___Gender__Person_in_Need_", \'Not Asked\', \'Other/Unknown/Cannot determine\');'
sqlScript38 = 'UPDATE "TOTAL_SATURNCLOUD" SET "Demographics___Gender__Person_in_Need_"=REPLACE("Demographics___Gender__Person_in_Need_", \'Declined to Answer\', \'Other/Unknown/Cannot determine\');'
sqlScript39 = 'UPDATE "TOTAL_SATURNCLOUD" SET "Demographics___Gender__Person_in_Need_"=REPLACE("Demographics___Gender__Person_in_Need_", \'Trans\', \'Other/Unknown/Cannot determine\');'
sqlScript40 = 'UPDATE "TOTAL_SATURNCLOUD" SET "Demographics___Gender__Person_in_Need_"=REPLACE("Demographics___Gender__Person_in_Need_", \'Client is person in need (not applicable)\', \'Other/Unknown/Cannot determine\');'

# ---------------------------------------------------------------------------
# -- COUNT CALLREPORT BY DATEOFCALL AND TAXONOMYNAME AND DEMOGRAPHICS
# ---------------------------------------------------------------------------
sqlScript41 = 'CREATE OR REPLACE TABLE \"TOTAL_SATURNCLOUD\" AS SELECT \"DateOfCall\", \"TaxonomyCode\", \"TaxonomyName\", \"Demographics___Gender__Person_in_Need_\", \"Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_\", COUNT(*) AS \"NumberOfCalls\" FROM \"TOTAL_SATURNCLOUD\" GROUP BY \"DateOfCall\", \"TaxonomyCode\", \"TaxonomyName\", \"Demographics___Gender__Person_in_Need_\", \"Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_\";'

# ---------------------------------------------------------------------------
# -- CHANGE DATEOFCALL DATATYPE TO DATE
# ---------------------------------------------------------------------------
sqlScript42 = 'ALTER TABLE "TOTAL_SATURNCLOUD" ADD \"DATE\" DATE;'
sqlScript43 = 'CREATE OR REPLACE TABLE "TOTAL_SATURNCLOUD" AS(SELECT to_date("DateOfCall") date,"NumberOfCalls", "TaxonomyCode", "TaxonomyName", "Demographics___Gender__Person_in_Need_", "Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_" FROM "TOTAL_SATURNCLOUD");'
sqlScript44 = 'alter table "TOTAL_SATURNCLOUD" rename column "DATE" to "DateOfCall";'

In [ ]:
sqlList = [sqlScript4 ,sqlScript5, sqlScript6, sqlScript7, 
           sqlScript15, sqlScript23, sqlScript24 ,sqlScript25 ,sqlScript25 ,
           sqlScript26 ,sqlScript27, sqlScript28, sqlScript29,
           sqlScript30, sqlScript31 ,sqlScript32 ,sqlScript33 ,
           sqlScript34 ,sqlScript35, sqlScript36, sqlScript37,
           sqlScript38, sqlScript39 ,sqlScript40 ,sqlScript41 ,
           sqlScript42 ,sqlScript43, sqlScript44]
# sqlList

In [ ]:
for u in sqlList:
    try:
        conn.cursor().execute(u)
    except: 
        print(f'This SQL Query Could Not Be Executed:\n{u}')
        
print('Preprocessing Completed')

# Predictive Modeling & Upload

In [ ]:
for t in sqlList[:3]:
    try:
        conn.cursor().execute(t)
    except: 
        print(f'This SQL Query Could Not Be Executed:\n{t}')
        
print('Snowflake Setup Complete')

In [ ]:
# Creates dataframe frome Snowflake SQL Table
curr = conn.cursor()
sql = 'SELECT * FROM TOTAL_SATURNCLOUD'
curr.execute(sql)

df = curr.fetch_pandas_all()

### Renaming and dropping columns

In [ ]:
df.rename(columns={'TaxonomyCode':'Level5Code',
'Demographics___Gender__Person_in_Need_':'Gender',
'Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_':'Militants'},inplace=True)

df['Level2Code'] = df['Level5Code'].str.extract('(R.)', expand=True)

df = df.drop(columns  = ['Level5Code','TaxonomyName'])

### Encoding Militants Column

In [ ]:
le = LabelEncoder()

## Militants - Categorical to Numerical
taxonomy_encoded = le.fit_transform(df['Militants'])
df['Militants'] = taxonomy_encoded

df3 = pd.get_dummies(df,columns=['Level2Code','Gender'])

df3 = df3.groupby(df3['DateOfCall'], as_index=False, sort=True).sum()

### Basic number of unqiue values assessment

In [ ]:
genderList = df['Gender'].unique()
# for i in genderList:
#     print("Number of " + i + " :"+ str(len(df[df['Gender'] == i])))

In [ ]:
MilitantList = df['Militants'].unique()
# for i in MilitantList:
#     print("Number " + str(i) + " :" + str(len(df[df['Militants'] == i])))

In [ ]:
# df3.shape

In [ ]:
# Describe all the data
# df3.describe(include='all')

In [ ]:
# summarize missing by column
# df3.isna().sum()

In [ ]:
## Separate dates for future plotting
train_dates = pd.to_datetime(df3['DateOfCall'])

## Variables for training
df4 = df3.iloc[:, 1:11].astype(float)
#df4.dtypes

In [ ]:
## Normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(df4)
df4_scaled = scaler.transform(df4)

In [ ]:
## Setting up a training/validation/testing dataset
X_train = []
y_train = []

n_future = 1 # Number of days we want to predict into the future
n_past = 14 # Number of past days we want to use to predict the future

for i in range (n_past, len(df4_scaled) - n_future +1):
    X_train.append(df4_scaled[i - n_past:i, 0:df4.shape[1]])
    y_train.append(df4_scaled[i + n_future - 1:i + n_future, 0])

## Convert to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

## Build a LSTM complex model with dropout, add L1 regularization, smaller weight. 

In [ ]:
# from tensorflow.keras.regularizers import l1
l1_penalty = 0.001

# build a two layer neural network with regularization
def build_model5():
    model = Sequential()
    model.add(LSTM(64,
                   activation = 'relu',
                   kernel_regularizer=l1(l1_penalty),
                   input_shape = (X_train.shape[1], X_train.shape[2]),
                   return_sequences = True))
    model.add(LSTM(64,
                   activation = 'relu',
                   kernel_regularizer=l1(l1_penalty),
                   return_sequences = True))
    model.add(Dropout(0.5))
    model.add(LSTM(64,
                   activation = 'relu',
                   kernel_regularizer=l1(l1_penalty),
                   return_sequences = True))
    model.add(Dropout(0.5))
    model.add(LSTM(64,
                   activation = 'relu',
                   kernel_regularizer=l1(l1_penalty),
                   return_sequences = False))
    model.add(Dropout(0.5))
    model.add(Dense(y_train.shape[1]))
    
    model.compile(optimizer = 'adam',
                  loss = 'mse',
                  metrics = ['mae'])
    return model

print('The model will take a few minutes to train.')
# fit this model/architecture to my data
history5 = build_model5()
history5.fit(X_train,
          y_train,
          epochs = 500,
          batch_size = 32,
          validation_split = 0.2,
          verbose = 0)
print('Training completed.')

In [ ]:
#summarize model
# history5.summary()

In [ ]:
# plot the loss function per epoch
# plt.plot(history5.history.history['loss'],
#          color='red')
# plt.plot(history5.history.history['val_loss'],
#          color='blue')

In [ ]:
# plot the loss function per epoch
# plt.plot(history5.history.history['mae'],
#          color='red')
# plt.plot(history5.history.history['val_mae'],
#          color='blue')

In [ ]:
# print('The min validation loss of',
#       np.min(history5.history.history['val_loss']),
#       ',\n was at epoch',
#       np.argmin(history5.history.history['val_loss']))

# print('The min validation mae of',
#       np.min(history5.history.history['val_mae']),
#       ',\n was at epoch',
#       np.argmin(history5.history.history['val_mae']))

## Forecasting Future Number Of Calls

In [ ]:
# Begin with the next day in training and forcast future
future_ndays = 30
theVarName = list(train_dates)[-1]
theVarName = theVarName + timedelta(days=1)

forecast_period_dates = pd.date_range(theVarName, 
                                      periods = future_ndays, 
                                      freq = '1d').tolist()
## forecast
forecast = history5.predict(X_train[-future_ndays:])

## Perform inverse transformation to rescale back to original range
forecast_copies = np.repeat(forecast, 
                            df4.shape[1], 
                            axis = -1)
y_pred_future = scaler.inverse_transform(forecast_copies)[:, 0]

## Convert forecast timestamp to date
forecast_dates = []
for time_i in forecast_period_dates:
    forecast_dates.append(time_i.date())

df_forecast = pd.DataFrame({'DateOfCall': np.array(forecast_dates), 
                            'Forecast NumberOfCalls':y_pred_future})
df_forecast['DateOfCall'] = pd.to_datetime(df_forecast['DateOfCall'])

#df_forecast

In [ ]:
# df_forecast

In [ ]:
# Original = df3[['DateOfCall', 'NumberOfCalls']]
# Original['DateOfCall'] = pd.to_datetime(Original['DateOfCall'])
# Original = Original.loc[Original['DateOfCall'] >= '2021-01-01']

## Plot Observed and forecast
# sns.lineplot(Original['DateOfCall'], Original['NumberOfCalls'])
# sns.lineplot(df_forecast['DateOfCall'], df_forecast['Forecast NumberOfCalls'])

### Upload Predictions to Table In Same Snowflake Database

In [ ]:
# Ensures Column Names are in the Correct Format for Snowflake
def moreFormat(theInput):
        
        anthrList = []
        stringList = []
        o = 0
        
        for i in theInput:
            if i.startswith(' '):
                theInput[o] = i.strip()
            o = o + 1
            
            if len(i) > 255:
                i = i[:255]
        
            splChar = [
            '(',')','[',']','/','\\','-','.',':','\t','<','>','"','?',
            '!','`',"'",'&','^',' ','{','}','+','=','~','|',';','%','#']
            for x in splChar:
                i = i.replace(x, '_')
            anthrList.append(i)
            stringList.append(i)

        stringList = str(stringList)
        stringList = stringList.replace("'", '"')
        stringList = stringList.replace(",", ' varchar,\n')
        stringList = stringList.strip("[]")
        stringList = stringList + " varchar"

        return anthrList, stringList

In [ ]:
# df_forecast.convert_dtypes()
thisVar1 = df_forecast.columns.tolist()
df_forecast.columns, stringList1 = moreFormat(thisVar1)

# Snowflake not accepting pandas datetime datatype correctly
df_forecast['DateOfCall'] = df_forecast['DateOfCall'].astype('string')

In [ ]:
# df_forecast.dtypes

In [ ]:
# SQL for Table Column Names and Datatypes
stringList1 = '"DateOfCall" VARCHAR, "Forecast_NumberOfCalls" FLOAT'

In [ ]:
conn.cursor().execute("USE DATABASE {}".format(dbName.upper()))

# try:
#     conn.cursor().execute("DROP TABLE \"TOTAL_SATURNCLOUD_PREDICTIONS\"")
# except:
#     continue
    
# Create Snowflake Table and Columns & Upload the DF to SF
conn.cursor().execute(
    "CREATE OR REPLACE TABLE "
    "TOTAL_SATURNCLOUD_PREDICTIONS({})".format(stringList1))
success, nchunks, nrows, _ = write_pandas(conn, df_forecast, 'TOTAL_SATURNCLOUD_PREDICTIONS')

#### Changes VARCHAR to DATE

In [ ]:
conn.cursor().execute('ALTER TABLE "TOTAL_SATURNCLOUD_PREDICTIONS" ADD "DATE" DATE;')


conn.cursor().execute(
    'CREATE OR REPLACE TABLE \"TOTAL_SATURNCLOUD_PREDICTIONS\" AS(SELECT to_date(\"DateOfCall\") date,\"Forecast_NumberOfCalls\"FROM \"TOTAL_SATURNCLOUD_PREDICTIONS\");')

conn.cursor().execute('alter table \"TOTAL_SATURNCLOUD_PREDICTIONS\" rename column \"DATE\" to \"DateOfCall\";')

### Select the past 30 days data to be compared to the forecasted calls
This is to make things smoother for PBI.

In [ ]:
conn.cursor().execute('ALTER TABLE \"TOTAL_SATURNCLOUD_PREDICTIONS\" ADD \"NumberOfCalls\" NUMBER(18,0);')
conn.cursor().execute('INSERT INTO \"TOTAL_SATURNCLOUD_PREDICTIONS\"(\"DateOfCall\", \"NumberOfCalls\") SELECT \"DateOfCall\", SUM(\"NumberOfCalls\") FROM \"TOTAL_SATURNCLOUD\" GROUP BY \"DateOfCall\" ORDER BY \"DateOfCall\" DESC LIMIT 30;')

## Sets Up Shares on Snowflake

In [ ]:
try:
    conn.cursor().execute(aSQLstatement)
    conn.cursor().execute(sqlScript)
    print('Share Created!')
except:
    print("This share already esixts in your Snowflake account.")

In [ ]:
try:
    conn.cursor().execute(aSQLstatement)
    conn.cursor().execute(sqlScript1)
    conn.cursor().execute(sqlScript2)
    conn.cursor().execute(sqlScript3)
except:
    print("There was an error in the grant usage on... SQL.")

In [ ]:
try:
    for x in ACCOUNT_LIST:
        sqlScriptShare = 'alter share {} add accounts={};'.format(shareName, x)
        conn.cursor().execute(aSQLstatement)
        try:
            conn.cursor().execute(sqlScriptShare)
        except:
            print('Account {} Could Not be Added.'.format(x))
    print("Accounts Successfully Added to {}!".format(shareName))
except:
    print("There was an error in the alter share {} add accounts=... SQL.".format(shareName))

In [ ]:
conn.cursor().close()
print("Connection to Snowflake has been closed.")

## Check your Snowflake account to see if the Database & Table are created and filled

## *IMPORTANT*: This only works with accounts that are from the same region and cloud!

In [ ]:
# Alternative Method
# from snowflake.connector.pandas_tools import pd_writer


# # Specify that the to_sql method should use the pd_writer function
# # to write the data from the DataFrame to the table named "customers"
# # in the Snowflake database.
# dfTotal.to_sql('FORMATTED_REPORTS', conn, index=False, method=pd_writer)